In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
wb = 'model_data.xlsx'

In [3]:
# Distance matrix d_ij
dist = pd.read_excel(wb, sheet_name='distance', index_col=0)

In [4]:
sup_df = pd.read_excel(wb, sheet_name='capacity', index_col=0)
cap = sup_df['capacity']
L = 1000

In [5]:
# Demand-side data
dem_df = pd.read_excel(wb, sheet_name='demand', index_col=0)
demand  = dem_df['demand']           # D_j

In [6]:
c   = 0.000001           # $ per unit-mile  
TC  = 22_000          # units a truck can haul
B   = 1_500_000       # budget ceiling   

In [7]:
I = dist.index.tolist()    # supply nodes
J = dist.columns.tolist()  # demand nodes

In [8]:
m = gp.Model('LocationTransport')

Set parameter Username
Set parameter LicenseID to value 2673805
Academic license - for non-commercial use only - expires 2026-05-30


In [9]:
# Decision variables
x = m.addVars(I, J, name='x', lb=0.0)                    # shipped units
z = m.addVars(I, J, vtype=GRB.INTEGER, name='z', lb=0)   # trucks
y = m.addVars(I, vtype=GRB.BINARY, name='y')             # process installed?

In [10]:
# Objective: minimise total shipping + setup cost
ship_cost = gp.quicksum(c * dist.loc[i, j] * x[i, j] for i in I for j in J)
setup_cost = gp.quicksum(L * y[i] for i in I)
m.setObjective(ship_cost + setup_cost, GRB.MINIMIZE)

In [11]:
# -----------------------------
# 3. CONSTRAINTS
# -----------------------------
# 3.1 Budget (total spend cannot exceed B)
m.addConstr(ship_cost + setup_cost <= B, name='Budget')

# 3.2 Supplier capacity (only active if y_i = 1)
for i in I:
    m.addConstr(gp.quicksum(x[i, j] for j in J) <= cap[i] * y[i],
                name=f'SupplyCap[{i}]')

# 3.3 Demand satisfaction
for j in J:
    m.addConstr(gp.quicksum(x[i, j] for i in I) >= demand[j],
                name=f'Demand[{j}]')

# 3.4 Truck capacity
for i in I:
    for j in J:
        m.addConstr(x[i, j] <= TC * z[i, j],
                    name=f'TruckCap[{i},{j}]')


m.Params.TimeLimit = 600          # stop after 10 min
#m.Params.MIPGap    = 0.006         # or stop at 2 % gap
#m.Params.Threads   = 0            # all available cores
#m.Params.MIPFocus  = 1            # improve bound, or 2 to find incumbents


# -----------------------------
# 4. SOLVE
# -----------------------------
m.optimize()

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 70618 rows, 140442 columns and 350842 nonzeros
Model fingerprint: 0x1b0c055d
Variable types: 69958 continuous, 70484 integer (526 binary)
Coefficient statistics:
  Matrix range     [2e-06, 2e+06]
  Objective range  [2e-06, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 6e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 221798.98269
Presolve removed 69958 rows and 69958 columns
Presolve time: 0.02s
Presolved: 660 rows, 70484 columns, 210926 nonzeros
Variable types: 69958 continuous, 526 integer (526 binary)
Performing another presolve...
Presolve time: 0.13s

Starting sifting (using dual s

In [12]:
# -----------------------------
# 5. REPORT RESULTS
# -----------------------------
if m.SolCount > 0:
    print(f'\nOptimal total cost: ${m.ObjVal:,.2f}')
    print('\nOpened supply nodes:')
    for i in I:
        if y[i].X > 0.5:
            shipped = sum(x[i, j].X for j in J)
            print(f'  {i}: installed (total units shipped {shipped:.1f})')
    print('\nFlows (i -> j):')
    for i in I:
        for j in J:
            if x[i, j].X > 1e-6:  # ignore tiny values
                print(f'  {i} -> {j}: {x[i,j].X:.1f} units, {z[i,j].X} trucks')


Optimal total cost: $124,599.12

Opened supply nodes:
  origin_1: installed (total units shipped 2000000.0)
  origin_106: installed (total units shipped 2000000.0)
  origin_108: installed (total units shipped 2000000.0)
  origin_110: installed (total units shipped 2000000.0)
  origin_111: installed (total units shipped 2000000.0)
  origin_114: installed (total units shipped 2000000.0)
  origin_133: installed (total units shipped 2000000.0)
  origin_134: installed (total units shipped 2000000.0)
  origin_164: installed (total units shipped 2000000.0)
  origin_165: installed (total units shipped 2000000.0)
  origin_166: installed (total units shipped 2000000.0)
  origin_167: installed (total units shipped 2000000.0)
  origin_168: installed (total units shipped 2000000.0)
  origin_169: installed (total units shipped 2000000.0)
  origin_170: installed (total units shipped 2000000.0)
  origin_171: installed (total units shipped 2000000.0)
  origin_172: installed (total units shipped 200000

In [13]:
print(f"python")

python


In [14]:
print(f"what the hell")

what the hell


In [16]:
print(f"gurobi20")

gurobi20


In [20]:
print(f"we are the best")


we are the best


In [25]:
print(f"I am working on it")
print(f"potka re ami valobasi")
print(f"potka")

I am working on it
potka re ami valobasi
potka
